In [ ]:
from pyincore import IncoreClient, Dataset, DataService, FragilityService, MappingSet
from pyincore.analyses.equitymetric import EquityMetric, EquityMetricUtil
from pyincore.analyses.buildingstructuraldamage import BuildingStructuralDamage
from pyincore.analyses.socialvulnerabilityscore import SocialVulnerabilityScore
from pyincore.analyses.housingrecoverysequential import HousingRecoverySequential
from pyincore.analyses.buildingnonstructuraldamage import BuildingNonStructDamage
from pyincore.analyses.combinedwindwavesurgebuildingdamage import CombinedWindWaveSurgeBuildingDamage
from pyincore.analyses.populationdislocation import PopulationDislocation

from pyincore_data.censusutil import CensusUtil

In [ ]:
client = IncoreClient()
datasvc = DataService(client)

## Prepare recovery time as scarce resource

### 1. Galveston Combined damage

In [ ]:
hazard_type = "hurricane"

# Galveston Hurricane Ike
hazard_id = "5fa5a228b6429615aeea4410"

# Galveston inventory data
# bldg_dataset_id = "63053ddaf5438e1f8c517fed" # Full city
bldg_dataset_id = "63ff6b135c35c0353d5ed3ac"  # Island Only

In [ ]:
# Building Wind Fragility mapping
mapping_id = "62fef3a6cef2881193f2261d"
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
# wind building damage
w_bldg_dmg = BuildingStructuralDamage(client)
w_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
w_bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)
w_bldg_dmg.set_parameter("result_name", "Galveston-wind-dmg")
w_bldg_dmg.set_parameter("hazard_type", hazard_type)
w_bldg_dmg.set_parameter("hazard_id", hazard_id)
w_bldg_dmg.set_parameter("num_cpu", 8)
w_bldg_dmg.run_analysis()

In [ ]:
# Surge-wave mapping
mapping_id = "6303e51bd76c6d0e1f6be080"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
# surge-wave building damage
sw_bldg_dmg = BuildingStructuralDamage(client)
sw_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
sw_bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)
sw_bldg_dmg.set_parameter("result_name", "Galveston-sw-dmg")
sw_bldg_dmg.set_parameter("hazard_type", hazard_type)
sw_bldg_dmg.set_parameter("hazard_id", hazard_id)
sw_bldg_dmg.set_parameter("num_cpu", 8)
sw_bldg_dmg.run_analysis()

In [ ]:
# flood mapping
mapping_id = "62fefd688a30d30dac57bbd7"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
# flood building damage
f_bldg_dmg = BuildingNonStructDamage(client)
f_bldg_dmg.load_remote_input_dataset("buildings", bldg_dataset_id)
f_bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)
f_bldg_dmg.set_parameter("result_name", "Galveston-flood-dmg")
f_bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
f_bldg_dmg.set_parameter("hazard_type", hazard_type)
f_bldg_dmg.set_parameter("hazard_id", hazard_id)
f_bldg_dmg.set_parameter("num_cpu", 8)
f_bldg_dmg.run_analysis()

In [ ]:
surge_wave_damage = sw_bldg_dmg.get_output_dataset("ds_result")
wind_damage = w_bldg_dmg.get_output_dataset("ds_result")
flood_damage = f_bldg_dmg.get_output_dataset("result")
combined_bldg_dmg = CombinedWindWaveSurgeBuildingDamage(client)
result_name = "Galveston-combined-dmg"
combined_bldg_dmg.set_input_dataset("surge_wave_damage", surge_wave_damage)
combined_bldg_dmg.set_input_dataset("wind_damage", wind_damage)
combined_bldg_dmg.set_input_dataset("flood_damage", flood_damage)
combined_bldg_dmg.set_parameter("result_name", result_name)
combined_bldg_dmg.run_analysis()

In [ ]:
combined_dmg = combined_bldg_dmg.get_output_dataset("ds_result")
combined_dmg_df = combined_dmg.get_dataframe_from_csv(low_memory=False)

### 2. Population Dislocation

In [ ]:
pop_dis = PopulationDislocation(client)
value_loss = "60354810e379f22e16560dbd"
bg_data = "603545f2dcda03378087e708"
hua_result_id = "63ee615d62b9d001e6a2b660" # city
# hua_result_id = "63ff8e895367c2261b4cb2ef"  # island

pop_dis.load_remote_input_dataset("block_group_data", bg_data)
pop_dis.load_remote_input_dataset("value_loss_param", value_loss)
pop_dis.load_remote_input_dataset("housing_unit_allocation", hua_result_id)

pop_dis.set_input_dataset("building_dmg", combined_dmg)

result_name = "galveston-pop-disl-results"
seed = 1111

pop_dis.set_parameter("result_name", result_name)
pop_dis.set_parameter("seed", seed)
pop_dis.run_analysis()

In [ ]:
population_dislocation_result = pop_dis.get_output_dataset("result")
population_dislocation_result_df = population_dislocation_result.get_dataframe_from_csv(
    low_memory=False
)
population_dislocation_result_df.head()

### 3. Run social vulnerability score analysis

In [ ]:
# get fips code to use fetch census data
fips = CensusUtil.get_fips_by_state_county(state="texas", county="galveston")
state_code = fips[:2]
county_code = fips[2:]

navs = CensusUtil.national_ave_values(year=2020)
national_vulnerability_feature_averages = Dataset.from_csv_data(navs, name="national_vulnerability_feature_averages.csv",
                                                                 data_type="incore:socialVulnerabilityFeatureAverages")

geo_type = "block%20group:*"
# geo_type = "tract:*"
social_vulnerability_dem_factors_df = CensusUtil.demographic_factors(state_code, county_code, year=2020,
                                                                      geo_type=geo_type)

# Temp fix: remove bad data point
social_vulnerability_dem_factors_df = social_vulnerability_dem_factors_df.dropna()

social_vulnerability_dem_factors = Dataset.from_dataframe(social_vulnerability_dem_factors_df,
                                                           name="social_vunlnerability_dem_factors",
                                                           data_type="incore:socialVulnerabilityDemFactors")

# Configure pyincore access and the analysis with the information above
social_vulnerability = SocialVulnerabilityScore(client)

social_vulnerability.set_parameter("result_name", "social_vulnerabilty")
social_vulnerability.set_input_dataset("national_vulnerability_feature_averages", national_vulnerability_feature_averages)
social_vulnerability.set_input_dataset("social_vulnerability_demographic_factors", social_vulnerability_dem_factors)

# Run social vulnerability damage analysis
result = social_vulnerability.run_analysis()
sv_result = social_vulnerability.get_output_dataset("sv_result")

### 4. Housing Household recovery analysis

In [ ]:
# Population dislocation result aggregated to the block group level.
population_dislocation = "60f5ea5d544e944c3cec66dd" # Galveston testbed
# Transition probability matrix per social vulnerability level.
transition_probability_matrix = "60f5e2ae544e944c3cec0794"
# Initial mass probability function for stage 0.
initial_probability_vector = "60f5e918544e944c3cec668b"

# Create housing recovery instance
housing_recovery = HousingRecoverySequential(client)

# Load input datasets
housing_recovery.set_input_dataset("population_dislocation_block", population_dislocation_result)
housing_recovery.load_remote_input_dataset("tpm", transition_probability_matrix)
housing_recovery.load_remote_input_dataset("initial_stage_probabilities", initial_probability_vector)

# Chain with Social Vulnerability output
housing_recovery.set_input_dataset('sv_result', sv_result)

# Set analysis parameters
housing_recovery.set_parameter("result_name", "housing_recovery_result")
housing_recovery.set_parameter("seed", 1234)
housing_recovery.set_parameter("t_delta", 1.0)
housing_recovery.set_parameter("t_final", 90.0)

housing_recovery.run()

In [ ]:
# Retrieve result dataset
hhrs_result = housing_recovery.get_output_dataset("ds_result")
recovery_df = hhrs_result.get_dataframe_from_csv()

### 5. Prepare recovery time as scarce resource

In [ ]:
scarce_resource_df = EquityMetricUtil.prepare_return_time_as_scarce_resource(
    recovery_df
)
scarce_resource = Dataset.from_dataframe(
    scarce_resource_df, "scarce_resource", data_type="incore:scarceResource"
)

In [ ]:
scarce_resource_df.head()

## Run Equity Metric Analysis

In [ ]:
equity_metric = EquityMetric(client)
equity_metric.set_parameter("result_name", "Galveston_recovery_time")
equity_metric.set_parameter("division_decision_column", "SVI")
equity_metric.load_remote_input_dataset(
    "housing_unit_allocation", "63ee615d62b9d001e6a2b660"
)
equity_metric.set_input_dataset("scarce_resource", scarce_resource)
em_result = equity_metric.run_analysis()

In [ ]:
equity_metric_df = equity_metric.get_output_dataset("equity_metric").get_dataframe_from_csv()
equity_metric_df.head()